# NOTEBOOK PARA REALIZAR TODO EL ANÁLISIS DE SECUENCIACIÓN, ENSAMBLE Y ANÁLISIS DE GENOMAS DE VIRUS DE INFLUENZA



## LIMPIEZA DE ARCHIVOS FASTQ

Cambiar el directorio a la carpeta a procesar. Ya debe contener los archivos FASTQ a procesar.

In [2]:
cd /home/admcenapa/DAVID_RENDON/TRABAJO/2025/CPA_TEST
pwd

/home/admcenapa/DAVID_RENDON/TRABAJO/2025/CPA_TEST


Crear una carpeta para depositar las secuencias crudas. Ahí se moverán los archivos crudos.

In [3]:
mkdir RAW
mv CPA* ./RAW

Crear ligas simbolicas a esos archivos en la carpeta actual para facilitar operaciones futuras.

In [4]:
ln -s ./RAW/CPA-00245-25-P1-1_S2_L001_R1_001.fastq.gz R1.fq.gz
ln -s ./RAW/CPA-00245-25-P1-1_S2_L001_R2_001.fastq.gz R2.fq.gz

Crear un directorio para depositar el análisis de calidad de lecturas crudas:

In [6]:
mkdir CALIDAD_CRUDA

Ejecutar el análisis de calidad. Usar ambiente calidad

In [71]:
conda activate calidad

In [9]:
fastqc R1.fq.gz R2.fq.gz --outdir ./CALIDAD_CRUDA --threads 14 

application/gzip
application/gzip
Started analysis of R1.fq.gz
Started analysis of R2.fq.gz
Approx 5% complete for R1.fq.gz
Approx 5% complete for R2.fq.gz
Approx 10% complete for R1.fq.gz
Approx 10% complete for R2.fq.gz
Approx 15% complete for R1.fq.gz
Approx 15% complete for R2.fq.gz
Approx 20% complete for R1.fq.gz
Approx 25% complete for R1.fq.gz
Approx 20% complete for R2.fq.gz
Approx 25% complete for R2.fq.gz
Approx 30% complete for R1.fq.gz
Approx 30% complete for R2.fq.gz
Approx 35% complete for R1.fq.gz
Approx 35% complete for R2.fq.gz
Approx 40% complete for R1.fq.gz
Approx 40% complete for R2.fq.gz
Approx 45% complete for R1.fq.gz
Approx 45% complete for R2.fq.gz
Approx 50% complete for R1.fq.gz
Approx 50% complete for R2.fq.gz
Approx 55% complete for R1.fq.gz
Approx 55% complete for R2.fq.gz
Approx 60% complete for R1.fq.gz
Approx 60% complete for R2.fq.gz
Approx 65% complete for R1.fq.gz
Approx 65% complete for R2.fq.gz
Approx 70% complete for R1.fq.gz
Approx 70% complete

Revisar los archivos HTML de salida para identificar parámetros que ayuden a la limpieza. Esto se puede lograr transfiriendo los archivos HTML a una computadora local y visualizando en un navedador de internet.

Ejecutar el programa de limpieza. Ajustar parámetros de acuerdo a las necesidades. Cuidado al modificar los cores (ver documentación). Los resultados se guardarán en ./TRIMMING/


In [74]:
trim_galore --paired --retain_unpaired --gzip --fastqc --cores 6 --clip_R1 16 --clip_R2 16 --three_prime_clip_R1 2 --three_prime_clip_R2 2 --length 100 -o TRIMMING/ R1.fq.gz R2.fq.gz

Path to Cutadapt set as: 'cutadapt' (default)
Cutadapt seems to be working fine (tested command 'cutadapt --version')
Cutadapt version: 4.9
Could not detect version of Python used by Cutadapt from the first line of Cutadapt (but found this: >>>#!/bin/sh<<<)
Letting the (modified) Cutadapt deal with the Python version instead
pigz 2.8
Parallel gzip (pigz) detected. Proceeding with multicore (de)compression using 6 cores

igzip command line interface 2.31.0
igzip detected. Using igzip for decompressing

No quality encoding type selected. Assuming that the data provided uses Sanger encoded Phred scores (default)

Output directory TRIMMING/ doesn't exist, creating it for you...

Output will be written into the directory: /home/admcenapa/DAVID_RENDON/TRABAJO/2025/CPA_TEST/TRIMMING/


AUTO-DETECTING ADAPTER TYPE
Attempting to auto-detect adapter type from the first 1 million sequences of the first file (>> R1.fq.gz <<)

Found perfect matches for the following adapter sequences:
Adapter type	

In [75]:
conda deactivate

Revisar los archivos HTML de salida para verificar los resultados de la limpieza. Queda a consideración de la experiencia del usuario el quedar conforme con la limpieza realizada.

## ALINEAMIENTO DE LECTURAS CONTRA BASE DE DATOS DE INFLUENZA

Hacer directorio para los alineamientos:

In [76]:
mkdir -p ALINEAMIENTO/BWA/ALL_SEGMENTS_MAPPING

Crear ligas simbolicas a los archivos a usar:

In [77]:
ln -s TRIMMING/R1_val_1.fq.gz reads_r1_tr
ln -s TRIMMING/R2_val_2.fq.gz reads_r2_tr
ln -s TRIMMING/R1_unpaired_1.fq.gz reads_u1_tr
ln -s TRIMMING/R2_unpaired_2.fq.gz reads_u2_tr

Activar ambiente conda alineamiento

In [78]:
conda activate alineamiento

Hacer los alineamientos de las secuencias pareadas y no pareadas contra la base de datos viral preparada en la UASIP.

In [79]:
#Alineamiento de secuencias pareadas 
bwa-mem2 mem -t 20 /backup/DATABASES/UASIP/BWAMEM2/INFLUENZA/influenza_db.fna reads_r1_tr reads_r2_tr >ALINEAMIENTO/BWA/ALL_SEGMENTS_MAPPING/paired.sam 

#Alineamiento de secuencias no pareadas de R1
bwa-mem2 mem -t 20 /backup/DATABASES/UASIP/BWAMEM2/INFLUENZA/influenza_db.fna reads_u1_tr >ALINEAMIENTO/BWA/ALL_SEGMENTS_MAPPING/unpaired_r1.sam

#Alineamiento de secuencias no pareadas de R2
bwa-mem2 mem -t 20 /backup/DATABASES/UASIP/BWAMEM2/INFLUENZA/influenza_db.fna reads_u2_tr >ALINEAMIENTO/BWA/ALL_SEGMENTS_MAPPING/unpaired_r2.sam

Looking to launch executable "/home/admcenapa/anaconda3/envs/alineamiento/bin/bwa-mem2.sse42", simd = .sse42
Launching executable "/home/admcenapa/anaconda3/envs/alineamiento/bin/bwa-mem2.sse42"


-----------------------------
Executing in SSE4.2 mode!!
-----------------------------
* SA compression enabled with xfactor: 8
* Ref file: /backup/DATABASES/UASIP/BWAMEM2/INFLUENZA/influenza_db.fna
* Entering FMI_search
* Index file found. Loading index from /backup/DATABASES/UASIP/BWAMEM2/INFLUENZA/influenza_db.fna.bwt.2bit.64
* Reference seq len for bi-index = 3955021927
* sentinel-index: 980727211
* Count:
0,	1
1,	1120073114
2,	1977510964
3,	2834948814
4,	3955021927

* Reading other elements of the index from files /backup/DATABASES/UASIP/BWAMEM2/INFLUENZA/influenza_db.fna
* Index prefix: /backup/DATABASES/UASIP/BWAMEM2/INFLUENZA/influenza_db.fna
* Read 0 ALT contigs
* Done reading Index!!
* Reading reference genome..
* Binary seq file = /backup/DATABASES/UASIP/BWAMEM2/INFLUENZA/influenza_db.fna.0123
* Reference genome size: 3955021926 bp
* Done reading reference genome !!

------------------------------------------
1. Memory pre-allocation for Chaining: 2786.7754 MB
2. Memory pre-

Entrar al directorio donde se depositaron los resultados del alineamiento.

In [80]:
cd ALINEAMIENTO/BWA/ALL_SEGMENTS_MAPPING 

Formatear los archivos sam resultantes en un archivo bam ordenado.

In [81]:
samtools merge -u - paired.sam unpaired_r1.sam unpaired_r2.sam | samtools sort -o all_sorted.bam && rm paired.sam unpaired_r1.sam unpaired_r2.sam

[W::bam_merge_core2] No @HD tag found.


Regresar al directorio del proyecto:

In [82]:
cd ../../../

Crear ocho directorios para depositar las lecturas por el segmento al cual alinearon.

In [83]:
mkdir ALINEAMIENTO/BWA/S{1..8}

Ejecutar el siguiente programa. Este programa abre el archivo bam, elimina alineamientos con alineamientos secundarios, y obtiene las lecturas por cada segmento. Se analiza, por segmento, el formato CIGAR asociado de cada lectura. Se filtran aquellas secuencias que alinearon en al menos 70% de sus bases. Se guardan los nombres de las lecturas pertinentes y se extraen las secuencias de los archivos fastq limpios usando dichos nombres. Los archivos producidos se guardan en las ubicaciones pertinentes.

In [84]:
~/analisis_influenza/bin/separar_seqs_segmentos.sh

Procesando segmento 1


Extrayendo lecturas R1 del segmento 1
[INFO] 23640 patterns loaded from file
Extrayendo lecturas R2 del segmento 1
[INFO] 23640 patterns loaded from file
Extrayendo lecturas no pareadas de R1 del segmento 1
[INFO] 23640 patterns loaded from file
Extrayendo lecturas no pareadas de R2 del segmento 1
[INFO] 23640 patterns loaded from file
Procesando segmento 2
Extrayendo lecturas R1 del segmento 2
[INFO] 23611 patterns loaded from file
Extrayendo lecturas R2 del segmento 2
[INFO] 23611 patterns loaded from file
Extrayendo lecturas no pareadas de R1 del segmento 2
[INFO] 23611 patterns loaded from file
Extrayendo lecturas no pareadas de R2 del segmento 2
[INFO] 23611 patterns loaded from file
Procesando segmento 3
Extrayendo lecturas R1 del segmento 3
[INFO] 21462 patterns loaded from file
Extrayendo lecturas R2 del segmento 3
[INFO] 21462 patterns loaded from file
Extrayendo lecturas no pareadas de R1 del segmento 3
[INFO] 21462 patterns loaded from file
Extrayendo lecturas no pareadas de

## ENSAMBLE DE NOVO DE GENOMAS DE INFLUENZA

Activar ambiente conda de los ensambladores

In [86]:
conda activate ensamble

Activa una terminal tmux para que se encargue de enviar los procesos aunque salgamos del servidor

In [ ]:
tmux new -s ensamblaje

Pegar el siguiente código de instrucciones en la terminal tmux. Este paso está diseñado para ejecutarse sobre todos los segmentos virales. Es importante revisar cada archivo log de cada subcarpeta para verificar que se haya ejecutado todos los procesos, desde el kmer 11 al kmer 127. Se hace en terminal tmux para que el ciclo no caiga cuando se termine la conexion con el servidor.

In [ ]:
for i in {1..8}; do
    ID="S${i}"
    id="s${i}"
    LOG="./ENSAMBLE/SPADES/${ID}/log_11_127_${ID}"
    mkdir -p "./ENSAMBLE/SPADES/${ID}"

    TMP_SCRIPT=$(mktemp /tmp/run_${ID}_XXXXXX.sh)

    cat > "$TMP_SCRIPT" <<EOF
#!/bin/bash

{
    echo "===== Iniciando proceso para ${ID} ====="
    echo "Inicio: \$(date)"
} >> "$LOG"

~/analisis_influenza/bin/ensamble.sh \\
    -a "./ALINEAMIENTO/BWA/${ID}/${id}_reads_r1.fq.gz" \\
    -b "./ALINEAMIENTO/BWA/${ID}/${id}_reads_r2.fq.gz" \\
    -x "./ALINEAMIENTO/BWA/${ID}/${id}_reads_u1.fq.gz" \\
    -y "./ALINEAMIENTO/BWA/${ID}/${id}_reads_u2.fq.gz" \\
    -t 26 --kini 11 --kfin 127 -o ./ENSAMBLE/SPADES/${ID}/OUT_11_127 >> "$LOG" 2>&1

{
    echo "Fin: \$(date)"
    echo "===== Finalizó proceso para ${ID} ====="
    echo
} >> "$LOG"

rm -- "\$0"
EOF

    chmod +x "$TMP_SCRIPT"
    nohup "$TMP_SCRIPT" &
    wait
done

[1] 2748085
nohup: se descarta la entrada y se añade la salida a 'nohup.out'


Se puede cerrar sesion con Ctrl + B, luego suelta y presiona D. Para reconectar la terminal tmux usar tmux attach -t ensamblaje

Monitorear que el proceso haya concluido, revisando periodicamente los archivos log.

In [3]:
pwd

/home/admcenapa/DAVID_RENDON/TRABAJO/2025/CPA_TEST


In [2]:
cd /home/admcenapa/DAVID_RENDON/TRABAJO/2025/CPA_TEST